# Research Question
Our research question is how many goals FC Barcelona is going to score in a certain match providing its date, opponent team and whether it is a home or an away match.

# Current Form Of Data
The data is currently in the following form (The attributes we are interested in): date, home_team_api_id, away_team_api_id, home_team_goal and away_team_goal. Here is a visual example representation:

| date                | home_team_api_id | away_team_api_id | home_team_goal | away_team_goal |
| :-------------------| :--------------: | :--------------: | :------------: | :------------: |
| 2008-08-17 00:00:00 | 9987             | 9993             | 0              | 1              |
| 2015-09-23 00:00:00 | 10192            | 9931             | 4              | 3              |

# Desired Form Of Data
The desired form of data should be in the following form: date, opponent team id, barca goals and whether or it is a home or an away match. Here is a visual representation of how the data should look like:

| date      | opponent_team_id | barca_goals | home_away |
| :---------| :--------------: | :---------: | --------: |

where <strong>date</strong> is the desired year minus the min year in the data set  
<strong>opponent_team_id</strong> is the team id of the opponent team. The team_id can be found in the used dataset  
<strong>barca_goals</strong> is the number of goals scored by FC Barcelona in this match  
<strong>home_away</strong> is whether it is a home or an away match where home represents 0 and away represents 1

Here is an example of how the data could be represented

| date      | opponent_team_id | barca_goals | home_away |
| :---------| :--------------: | :---------: | --------: |
| 0         | 10281            | 6           | 0         |
| 8         | 7878             | 3           | 1         |

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

database = sqlite3.connect("database.sqlite")


# Get FC Barcelona's team_api_id
In order to get the matches where FC Barcelona was involved in we need its team_api_id.  
The table which is named <strong>team</strong> is the table we are going to use to get the team_api_id. Here is a visual example representation (Only the attributes we are interested in):

| team_api_id | team_long_name |
| :---------- | -------------: |
| 8634        | FC Barcelona   |

What we are going to do is we are going to select the teams where their team_long_name is FC Barcelona (Only FC Barcelona will be returned) then we are going to get its team_api_id


In [2]:
# Get the team_api_id of FC Barcelona
barca_df = pd.read_sql_query("SELECT * from team where team_long_name='FC Barcelona'", database)
barca_api_id = barca_df.team_api_id[0]

barca_api_id

8634

# Select only the attributes related to our research problem

We are going to reach the desired form of data by getting the home and away matches alone and then concatenating them at the end.

For the home matches we are going to get the <strong>date</strong>, <strong>away_team_api_id</strong> (which represents the opponent team so we are going to rename it as opponnent_team_id) and <strong>home_team_goal</strong> (which represents the number of goals FC Barcelona scored in the match so we are going to rename it as barca_goals)

For the away matches we are going to get the <strong>date</strong>, <strong>home_team_api_id</strong> (which represents the opponent team so we are going to rename it as opponnent_team_id) and <strong>away_team_goal</strong> (which represents the number of goals FC Barcelona scored in the match so we are going to rename it as barca_goals)

Using the where clause in SQL and barca_api_id we can get the home and away matches for FC Barcelona by searching the home_team_api_id and away_team_api_id.

In [3]:
## Select only the attributee relevant to our research question

# Concerning the matches where FC Barcelona was the home team, we need the date, and the away team (opponent)
# and the number of goals the home team (FC Barcelona) scored
home_select_clause = "date, away_team_api_id as 'opponent_team_id', home_team_goal as 'barca_goals'"

# Concerning the matches where FC Barcelona was the away team, we need the date, and the home team (opponent)
# and the number of goals the away team (FC Barcelona) scored
away_select_clause = "date, home_team_api_id as 'opponent_team_id', away_team_goal as 'barca_goals'"

# Select only the matches that FC Barcelona involved in as the home team and the away team seperately
home_where_clause = "home_team_api_id=" + str(barca_api_id)
away_where_clause = "away_team_api_id=" + str(barca_api_id)

# Putting things together

Using the <strong>SELECT</strong> and <strong>WHERE</strong> clauses we can obtain the home and away matches 

In [4]:
# return the home matches of FC Barcelona
home_matches_df = pd.read_sql_query("SELECT " + str(home_select_clause) +" from match where " + str(home_where_clause), database)
home_matches_df.head()

,date,opponent_team_id,barca_goals
0,2008-11-08 00:00:00,10281,6
1,2008-11-23 00:00:00,8305,1
2,2008-12-06 00:00:00,10267,4
3,2008-12-13 00:00:00,8633,2
4,2009-01-03 00:00:00,8661,3


In [5]:
# return the home matches of FC Barcelona
away_matches_df = pd.read_sql_query("SELECT " + str(away_select_clause) +" from match where " + str(away_where_clause), database)
away_matches_df.head()

,date,opponent_team_id,barca_goals
0,2008-08-31 00:00:00,8388,0
1,2008-11-16 00:00:00,8479,2
2,2008-11-29 00:00:00,8302,3
3,2008-12-21 00:00:00,10205,2
4,2009-01-11 00:00:00,8371,3


# Set the home_away attribute
Set the home_away attribute for the home and away dataframes where the home dataframe gets a value of 0 and the away dataframe gets a value of 1

In [6]:
# Create a new attribute named home_away where 0 is home and 1 is away and apply it on our two data frames
home_matches_df['home_away'] = 0
home_matches_df.head()

,date,opponent_team_id,barca_goals,home_away
0,2008-11-08 00:00:00,10281,6,0
1,2008-11-23 00:00:00,8305,1,0
2,2008-12-06 00:00:00,10267,4,0
3,2008-12-13 00:00:00,8633,2,0
4,2009-01-03 00:00:00,8661,3,0


In [7]:
away_matches_df['home_away'] = 1
away_matches_df.head()

,date,opponent_team_id,barca_goals,home_away
0,2008-08-31 00:00:00,8388,0,1
1,2008-11-16 00:00:00,8479,2,1
2,2008-11-29 00:00:00,8302,3,1
3,2008-12-21 00:00:00,10205,2,1
4,2009-01-11 00:00:00,8371,3,1


# Get minimum year in the dataset

In order to reach the desired form of data we need to get the minimum year in the dataset so that we can use it to manipulate the date attribute as described above. We are going to do that by returning all the dates from the dataset. We will then manipulate the it by getting the year only from date attribute. The date attribute is in the following form yyyy-mm-dd hh:mm:ss so we are going to extract the year by getting the first integer in the date attribute which is the yyyy. Then we are going to convert the string to a float and then to an integer. After we obtain the years in an integer form we can then get the minimum year.

In [8]:
# Get all the dates from the database
date_df = pd.read_sql_query("SELECT date from match", database)
date_df.head()

,date
0,2008-08-17 00:00:00
1,2008-08-16 00:00:00
2,2008-08-16 00:00:00
3,2008-08-17 00:00:00
4,2008-08-16 00:00:00


In [9]:
# Extracting the year from the date attribute
date_df["date"] = date_df["date"].str.extract('(\d+)', expand=False).astype(float).astype(int)
date_df.head()

,date
0,2008
1,2008
2,2008
3,2008
4,2008


In [10]:
# Obtaining the min year
min_year = date_df["date"].min()
min_year

2008

# Manipulate date attribute

After obtaining the min year, we can use it to manipulate the date attribute by subtracting every year by the min year obtained

In [11]:
# maniuplate the date attribute
home_matches_df["date"] = home_matches_df["date"].str.extract('(\d+)', expand=False).astype(float).astype(int) - min_year
away_matches_df["date"] = away_matches_df["date"].str.extract('(\d+)', expand=False).astype(float).astype(int) - min_year

home_matches_df.head()

,date,opponent_team_id,barca_goals,home_away
0,0,10281,6,0
1,0,8305,1,0
2,0,10267,4,0
3,0,8633,2,0
4,1,8661,3,0


# Concatenate home and away matches

In [12]:
# Concatenate home and away matches
all_matches_df = pd.concat([home_matches_df, away_matches_df])

all_matches_df

,date,opponent_team_id,barca_goals,home_away
0,0,10281,6,0
1,0,8305,1,0
2,0,10267,4,0
3,0,8633,2,0
4,1,8661,3,0
...,...,...,...,...
147,8,9783,8,1
148,8,8603,2,1
149,8,7878,3,1
150,7,9910,1,1
